<a href="https://colab.research.google.com/github/dlwndms0812/Artificial_Intelligence/blob/main/homework4_Adam_2019110701.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.7.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.7.0
     |████████████████████████████████| 412.3 MB 20 kB/s 
     |████████████████████████████████| 503 kB 48.9 MB/s 
     |████████████████████████████████| 3.8 MB 37.3 MB/s 
     |████████████████████████████████| 50 kB 7.0 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=a7af5162a48606edd98bc80c1c812bef5e5a9fee50a31adae733c8dc8e1579e7
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32

In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [ ]:
X=tf.placeholder(tf.float32, [None, 28, 28, 1])
Y=tf.placeholder(tf.float32, [None, 10])
keep_prob=tf.placeholder(tf.float32)

In [ ]:
#3*3인 컨볼루션 계층, 오른쪽과 아래쪽으로 한칸씩 움직이는 32개의 커널을 가진 컨볼루션 계층
W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
L1=tf.nn.conv2d(X,W1,strides=[1,1,1,1], padding='SAME')
L1=tf.nn.relu(L1)
#커널 크기가 2*2인 풀링계층
L1=tf.nn.max_pool(L1,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#3*3크기의 커널 64개로 구성된 컨볼루션 계층
W2=tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2=tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2=tf.nn.relu(L2)
#2*2크기의 풀링계층
L2=tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#직전의 풀링계층 크기가 7*7*64에 256개의 뉴런으로 연결=완전 연결 계층(fully connected layer)
W3=tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3=tf.reshape(L2, [-1, 7*7*64])
L3=tf.matmul(L3, W3)
L3=tf.nn.relu(L3)
L3=tf.nn.dropout(L3, keep_prob)

In [ ]:
#L3의 출력값 256개를 받아 최종 출력값인 0-9 레이블을 갖는 10개의 출력값을 만듦
W4=tf.Variable(tf.random_normal([256,10], stddev=0.01))
model=tf.matmul(L3,W4)

In [ ]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(cost) #AdamOptimizer 사용

In [ ]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

batch_size=100
total_batch=int(mnist.train.num_examples/batch_size)

for epoch in range(15):
  total_cost=0
  for i in range(total_batch):
    batch_xs, batch_ys=mnist.train.next_batch(batch_size)
    batch_xs=batch_xs.reshape(-1, 28, 28, 1)  #모델에 입력값을 전달하기 위해 MNIST 데이터를 28*28 형태로 재구성
    _, cost_val=sess.run([optimizer, cost], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.8}) #수치 0.8
    total_cost+=cost_val
  print('Epoch:', '%04d' % (epoch+1), 
        'Avg.cost=', '{:.3f}'.format(total_cost/total_batch))
print('최적화 완료!')

In [ ]:
is_correct=tf.equal(tf.argmax(model,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도:', sess.run(accuracy, feed_dict={X:mnist.test.images.reshape( -1, 28,28,1), Y:mnist.test.labels, keep_prob:1}))

정확도: 0.9882
